# 준비

In [1]:
import sys
IN_COLAB = 'google.colab' in sys.modules


## 패키지 설치

In [2]:
!pip install pandas numpy tqdm==4.43.0 bayesian-optimization lightgbm

Traceback (most recent call last):
  File "c:\program files (x86)\microsoft visual studio\shared\anaconda3_64\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\program files (x86)\microsoft visual studio\shared\anaconda3_64\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\Scripts\pip.exe\__main__.py", line 5, in <module>
  File "c:\program files (x86)\microsoft visual studio\shared\anaconda3_64\lib\site-packages\pip\__init__.py", line 1, in <module>
    from pip._internal.utils.typing import MYPY_CHECK_RUNNING
  File "c:\program files (x86)\microsoft visual studio\shared\anaconda3_64\lib\site-packages\pip\_internal\__init__.py", line 2, in <module>
    import pip._internal.utils.inject_securetransport  # noqa
ModuleNotFoundError: No module named 'pip._internal.utils.inject_securetransport'


## 라이브러리 임포트

In [4]:
import sys
import pandas as pd                         # 데이터 분석 라이브러리
# import numpy as np                          # 계산 라이브러리
# from tqdm.auto import tqdm                  # 진행바
# from sklearn.metrics import roc_auc_score   # AUC 스코어 계산
# from sklearn.model_selection import KFold   # K-fold CV    
# from bayes_opt import BayesianOptimization  # 베이지안 최적화 라이브러리  
# from functools import partial               # 함수 변수 고정
# import lightgbm as lgb                      # LightGBM 라이브러리
# import multiprocessing
# import gc
import importlib
import warnings                             
warnings.filterwarnings("ignore")           # 경고 문구 미표시
# tqdm.pandas()
print(pd.show_versions())


INSTALLED VERSIONS
------------------
commit           : None
python           : 3.6.5.final.0
python-bits      : 64
OS               : Windows
OS-release       : 10
machine          : AMD64
processor        : Intel64 Family 6 Model 158 Stepping 9, GenuineIntel
byteorder        : little
LC_ALL           : None
LANG             : None
LOCALE           : None.None

pandas           : 1.0.3
numpy            : 1.15.4
pytz             : 2018.4
dateutil         : 2.7.3
pip              : None
setuptools       : 39.1.0
Cython           : 0.28.2
pytest           : 3.5.1
hypothesis       : None
sphinx           : 1.7.4
blosc            : None
feather          : None
xlsxwriter       : 1.0.4
lxml.etree       : 4.2.1
html5lib         : 1.0.1
pymysql          : None
psycopg2         : None
jinja2           : 2.10
IPython          : 6.4.0
pandas_datareader: None
bs4              : 4.6.0
bottleneck       : 1.2.1
fastparquet      : None
gcsfs            : None
lxml.etree       : 4.2.1
matplotlib    

In [5]:
def reload_module(module):
    for _ in range(2):
        for k, v in sys.modules.items():
            if k.startswith(module.__name__):
                importlib.reload(v)

## 상수 정의

In [6]:
DATA_DIR = 'data'

## Colab 설정

In [7]:
if IN_COLAB:
    import getpass
    import urllib.parse

    github_repo = "github.com/Heo-H/dacon"
    branch_name = "dacon-202003"

    print("Please enter your Github account.")
    github_username = urllib.parse.quote(input("username:"))
    github_password = urllib.parse.quote(getpass.getpass("password:"))
    github_url = f"https://{github_username}:{github_password}@{github_repo}"
    github_sealed_url = f"https://{github_username}:****@{github_repo}"

    output = !git clone -b $branch_name --single-branch $github_url tmp_dacon_202003 && mv tmp_dacon_202003/* . && rm -r tmp_dacon_202003
    for line in output:
        print(line.replace(github_url, github_sealed_url))

    del github_repo
    del github_username
    del github_password
    del github_url
    del github_sealed_url
    del branch_name

In [8]:
if IN_COLAB:
    DRIVE_DIR = '/content/drive'
    from google.colab import drive
    drive.mount(DRIVE_DIR)

    import os
    os.symlink(f'{DRIVE_DIR}/My Drive/data', DATA_DIR)

# 학습 데이터

In [9]:
train = pd.read_csv('data/train.csv')

# 특징 추출

In [18]:
import sclearn.feature_engineering
reload_module(sclearn.feature_engineering)

# x_train = sclearn.feature_engineering.prepare_x_data(train[:1000000])
# y_train = sclearn.feature_engineering.prepare_y_data(train[:1000000])

# x_train = sclearn.feature_engineering.prepare_x_data(train)
x_train = sclearn.feature_engineering.chunkize_dataframe(sclearn.feature_engineering.prepare_x_data, train, n_splits=4)
y_train = sclearn.feature_engineering.prepare_y_data(train)

In [19]:
x_train.head()

,time,p0_species,p1_species,p0_event_Ability,p0_event_AddToControlGroup,p0_event_Camera,p0_event_ControlGroup,p0_event_GetControlGroup,p0_event_Right Click,p0_event_Selection,...,delta_first_time_(16E4) - BuildEvolutionChamber,delta_first_time_(1401) - BuildSiegeTank,delta_first_time_(4AE0) - BuildOracleStasisTrap,delta_first_time_(1821) - MorphZergling,delta_first_time_(1584) - TrainDarkTemplar,delta_first_time_(1540) - BuildNexus,delta_first_time_(15A2) - TrainCarrier,delta_first_time_(1406) - BuildBattleHellion,delta_first_time_(5640) - MorphToTransportOverlord,Cluster
0,444.0,0,0,34.0,2.0,444.0,0.0,24.0,35.0,50.0,...,0.0,99713.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13
1,599.0,1,0,77.0,1.0,627.0,0.0,162.0,160.0,186.0,...,0.0,99728.0,0.0,0.0,0.0,-99543.0,0.0,0.0,0.0,3
2,599.0,1,2,69.0,6.0,413.0,0.0,99.0,160.0,90.0,...,99689.0,0.0,-99599.0,99819.0,0.0,-99594.0,0.0,0.0,0.0,41
3,599.0,0,1,82.0,0.0,713.0,0.0,132.0,276.0,180.0,...,0.0,-99427.0,0.0,0.0,0.0,99433.0,0.0,0.0,0.0,35
4,343.0,0,2,57.0,1.0,430.0,0.0,224.0,177.0,67.0,...,0.0,0.0,0.0,99784.0,0.0,0.0,0.0,0.0,0.0,2


# 모델 구축

In [20]:
import sclearn.learning
import random
reload_module(sclearn.learning)
model_sets = []
random.seed(1)
for i in range(5):
    cur_rand = random.randint(0, 65535)
    print(cur_rand)
    models, lgb_bo = sclearn.learning.train_models(x_train, y_train, random_state=cur_rand)
    model_sets.append(models)

17611
|   iter    |  target   | baggin... | colsam... | featur... | learni... | n_esti... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------
|  1        |  0.7074   |  0.877    |  0.7754   |  0.9242   |  0.05462  |  224.0    |  86.66    |  4.771    |  49.27    |  0.4102   |
|  2        |  0.7093   |  0.6251   |  0.4365   |  0.5614   |  0.06011  |  599.7    |  208.6    |  1.616    |  12.95    |  0.02409  |
|  3        |  0.6905   |  0.5158   |  0.9339   |  0.7891   |  0.02477  |  144.8    |  87.57    |  6.959    |  41.73    |  0.4425   |
|  4        |  0.7044   |  0.5011   |  0.2721   |  0.6267   |  0.08175  |  836.4    |  189.2    |  5.748    |  0.09107  |  0.4427   |
|  5        |  0.7071   |  0.8933   |  0.9798   |  0.5972   |  0.02619  |  465.7    |  46.88    |  6.259    |  46.57    |  0.3395   |
|  6        |  0.7064   |  0.703    |  0.7749   |  0.676

|  1        |  0.706    |  0.7961   |  0.1978   |  0.7304   |  0.08751  |  493.1    |  199.2    |  1.667    |  47.64    |  0.02089  |
|  2        |  0.7084   |  0.6086   |  0.335    |  0.72     |  0.07989  |  453.5    |  98.46    |  1.394    |  20.89    |  0.1293   |
|  3        |  0.6932   |  0.5314   |  0.9112   |  0.6433   |  0.06717  |  86.33    |  36.28    |  5.249    |  42.78    |  0.4588   |
|  4        |  0.7112   |  0.5951   |  0.9901   |  0.6812   |  0.04278  |  583.2    |  69.19    |  3.144    |  11.59    |  0.457    |
|  5        |  0.6568   |  0.9533   |  0.1468   |  0.6552   |  0.003313 |  226.2    |  22.71    |  1.607    |  38.06    |  0.4852   |
|  6        |  0.7124   |  0.6946   |  0.629    |  0.5117   |  0.02345  |  599.4    |  74.76    |  3.855    |  38.78    |  0.1458   |
|  7        |  0.6878   |  0.5      |  0.0      |  0.5      |  0.0001   |  1.024e+0 |  256.0    |  1.382e-0 |  2.832e-0 |  0.4999   |
|  8        |  0.7092   |  0.6152   |  0.4053   |  0.9106   | 

# 테스트

In [21]:
test = pd.read_csv('data/test.csv')

In [22]:
import sclearn.feature_engineering
reload_module(sclearn.feature_engineering)

# x_test = sclearn.feature_engineering.prepare_x_data(test)
x_test = sclearn.feature_engineering.chunkize_dataframe(sclearn.feature_engineering.prepare_x_data, test, n_splits=4)

In [23]:
x_test.head()

,time,p0_species,p1_species,p0_event_Ability,p0_event_AddToControlGroup,p0_event_Camera,p0_event_ControlGroup,p0_event_GetControlGroup,p0_event_Right Click,p0_event_Selection,...,delta_first_time_(16E4) - BuildEvolutionChamber,delta_first_time_(1401) - BuildSiegeTank,delta_first_time_(4AE0) - BuildOracleStasisTrap,delta_first_time_(1821) - MorphZergling,delta_first_time_(1584) - TrainDarkTemplar,delta_first_time_(1540) - BuildNexus,delta_first_time_(15A2) - TrainCarrier,delta_first_time_(1406) - BuildBattleHellion,delta_first_time_(5640) - MorphToTransportOverlord,Cluster
38872,426.0,1,1,31.0,1.0,232.0,0.0,9.0,241.0,52.0,...,0.0,0.0,0.0,0.0,0.0,99718.0,0.0,0.0,0.0,29
38873,367.0,2,1,34.0,0.0,200.0,4.0,18.0,51.0,114.0,...,0.0,0.0,0.0,-99820.0,0.0,99893.0,0.0,0.0,0.0,49
38874,281.0,0,0,33.0,0.0,245.0,0.0,63.0,79.0,42.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40
38875,478.0,0,1,95.0,8.0,515.0,0.0,255.0,218.0,123.0,...,0.0,-99625.0,0.0,0.0,0.0,99877.0,0.0,0.0,0.0,5
38876,170.0,2,2,7.0,2.0,129.0,0.0,387.0,30.0,25.0,...,0.0,0.0,0.0,99837.0,0.0,0.0,0.0,0.0,0.0,30


In [24]:
import sclearn.learning
reload_module(sclearn.learning)

for i in range(5):
    submission = sclearn.learning.test_models(model_sets[i], x_test)
    submission.to_csv('submission_' + str(i) + '.csv')
    submission.head()

In [25]:
df_arr = []
for i in range(5):
    df = pd.read_csv('submission_' + str(i) + '.csv')
    df_arr.append(df)
    df_arr[i].head()

In [28]:
df final = pd.concat([])

,game_id,winner
0,38872,0.518079
1,38873,0.735222
2,38874,0.386884
3,38875,0.125060
4,38876,0.605323


In [34]:
for i in range(5):
    df_arr[i] = df_arr[i].rename(columns={'winner':'winner'+str(i)})

# submit = df_arr[0]
# submit = pd.concat([df_arr[0], df_arr[1][['winner']]], axis=1)
# submit
# for i in range(1, 5):
#     submit = submit.conca

In [39]:

submit = pd.concat([df_arr[0], df_arr[1][['winner1']], df_arr[2][['winner2']], df_arr[3][['winner3']], df_arr[4][['winner4']]], axis=1)
submit

,game_id,winner0,winner1,winner2,winner3,winner4
0,38872,0.438325,0.518079,0.569683,0.577401,0.505146
1,38873,0.867941,0.735222,0.701343,0.613010,0.656040
2,38874,0.434708,0.386884,0.417716,0.436444,0.443957
3,38875,0.089959,0.125060,0.151980,0.184807,0.142788
4,38876,0.551667,0.605323,0.619235,0.614321,0.600617
...,...,...,...,...,...,...
16782,55654,0.671801,0.630071,0.674081,0.663704,0.660027
16783,55655,0.113945,0.201043,0.199242,0.231664,0.205583
16784,55656,0.762480,0.720707,0.730283,0.743149,0.738786
16785,55657,0.692533,0.603034,0.634787,0.586707,0.599876


In [40]:
submit['winner'] = (submit['winner0']+submit['winner1']+submit['winner2']+submit['winner3']+submit['winner4'])/5
submit

,game_id,winner0,winner1,winner2,winner3,winner4,winner
0,38872,0.438325,0.518079,0.569683,0.577401,0.505146,0.521727
1,38873,0.867941,0.735222,0.701343,0.613010,0.656040,0.714711
2,38874,0.434708,0.386884,0.417716,0.436444,0.443957,0.423942
3,38875,0.089959,0.125060,0.151980,0.184807,0.142788,0.138919
4,38876,0.551667,0.605323,0.619235,0.614321,0.600617,0.598232
...,...,...,...,...,...,...,...
16782,55654,0.671801,0.630071,0.674081,0.663704,0.660027,0.659937
16783,55655,0.113945,0.201043,0.199242,0.231664,0.205583,0.190295
16784,55656,0.762480,0.720707,0.730283,0.743149,0.738786,0.739081
16785,55657,0.692533,0.603034,0.634787,0.586707,0.599876,0.623387


In [42]:
submit = submit.drop(['winner0','winner1','winner2','winner3','winner4'], axis=1)
submit

,game_id,winner
0,38872,0.521727
1,38873,0.714711
2,38874,0.423942
3,38875,0.138919
4,38876,0.598232
...,...,...
16782,55654,0.659937
16783,55655,0.190295
16784,55656,0.739081
16785,55657,0.623387


In [44]:
submit.to_csv('this_is_our_hope.csv',index=False)